In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Local (mac) orientation debug from a single TNG cutout + local group catalog.

- Orientation estimators (stars only):
    * SubhaloSpin (catalog, total)  J_tot
    * Stellar spin from cutout      J_star
    * PCA v3 (minor axis)           v3_hat
- Imaging uses STARS ONLY (to match your TNG reference figure). You can flip a flag
  to include gas visually if you want (doesn't affect the orientation calcs).

Saves:
  tests/56372_sightlines/tng_orientation_checks_debug/debug_sid<SID>_part-stars_snap<SNAP>.png
  tests/56372_sightlines/tng_orientation_checks_debug/orientation_sid<SID>_snap<SNAP>.json
  tests/56372_sightlines/tng_orientation_checks_debug/orientation_sid<SID>_snap<SNAP>.csv
"""

import os, json, math, h5py
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import illustris_python as il   # pip install illustris_python

# ────────────── USER PATHS (EDIT if your paths differ) ──────────────
CUTOUT_H5        = r"/Users/tsingh65/ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/cutout_ALLFIELDS_sphere_2p1Rvir_sub563732.hdf5"
GROUPCAT_BASE    = r"/Users/tsingh65/ASU Dropbox/Tanmay Singh/COS_GASS/TNG_group_cat_snap99"   # directory that contains groups_099.*
OUT_DIR          = r"/Users/tsingh65/github_repos/COS-GASS/tests/56372_sightlines/tng_orientation_checks_debug"

SNAP             = 99
SID              = 563732

# Imaging from stars only (to match your TNG plots). Set True to add gas visually.
INCLUDE_GAS_IN_IMAGES = False

# ────────────── Figure / style (match your TNG plot) ──────────────
part          = "stars"
extent_kpc_h  = 80.0
nbin          = 1200
REMOVE_BULK_VEL = False
use_rhalf_aperture = True
rhalf_multiplier   = 10.0

dpi_out        = 120
cmap_name      = "jet"
bg_color       = "#0f111a"
fg_color       = "#eaeaea"
spin_tot_color = "#06d7d9"
spin_star_color= "#ffb000"
nodes_color    = "#f37ea1"
axes_guide     = "#eaeaea"
scalebar_color = "#eaeaea"
VMIN = 4.0
VMAX = 7.0

mpl.rcParams.update({
    "figure.facecolor": bg_color, "axes.facecolor": bg_color, "savefig.facecolor": bg_color,
    "axes.edgecolor": fg_color, "axes.labelcolor": fg_color, "xtick.color": fg_color,
    "ytick.color": fg_color, "text.color": fg_color, "font.family": "DejaVu Sans", "font.size": 12,
})

# ────────────── Helpers ──────────────
def unit(v):
    v = np.asarray(v, float)
    n = np.linalg.norm(v)
    return v / n if n > 0 else v

def minimal_image_delta(dpos, box): 
    return (dpos + 0.5*box) % box - 0.5*box

def recenter_positions(x, center, box): 
    return minimal_image_delta(x - center[None,:], box)

def rodrigues_u_to_v(u, v):
    """Active rotation mapping u -> v; returns R so that R @ u == v."""
    u = unit(u); v = unit(v)
    c = float(np.clip(np.dot(u, v), -1.0, 1.0))
    if c > 1 - 1e-12:
        return np.eye(3)
    if c < -1 + 1e-12:
        a = unit(np.cross(u, [1,0,0]) if abs(u[0]) < 0.9 else np.cross(u, [0,1,0]))
        K = np.array([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]])
        return np.eye(3) + 2*(K @ K)    # π-rotation
    a = np.cross(u, v); s = float(np.linalg.norm(a))
    a = a / s
    K = np.array([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]])
    return np.eye(3) + K*s + (K @ K)*(1 - c)

def inc_PA_from_vector(J):
    """
    inc (deg): inclination wrt +z (0 face-on, 90 edge-on), uses |cos|.
    PA  (deg): position angle of line of nodes (z × J) in [0,180), from +x.
    """
    J = np.asarray(J, float); Jn = np.linalg.norm(J)
    if Jn == 0: return np.nan, np.nan
    Jhat = J / Jn
    inc = np.degrees(np.arccos(np.clip(abs(Jhat[2]), 0, 1)))
    n = np.cross([0,0,1.0], Jhat)
    PA = 0.0 if np.hypot(n[0], n[1]) < 1e-14 else (np.degrees(np.arctan2(n[1], n[0])) % 180.0)
    return inc, PA

def pca3_weighted(X, w):
    """Mass-weighted 3D PCA. Return eigvals(desc), eigvecs(cols), and COM."""
    X = np.asarray(X, float); w = np.asarray(w, float)
    wsum = np.sum(w); xc = np.sum(X*w[:,None], axis=0) / max(wsum, 1e-30)
    X0 = X - xc
    C = (X0*w[:,None]).T @ X0 / max(wsum, 1e-30)
    evals, evecs = np.linalg.eigh(C)
    idx = np.argsort(evals)[::-1]
    return evals[idx], evecs[:, idx], xc

def pca2_weighted(XY, w):
    """Mass-weighted 2D PCA on projected coords; return major-axis unit vector and angle (deg)."""
    XY = np.asarray(XY, float); w = np.asarray(w, float)
    wsum = np.sum(w); xc = np.sum(XY*w[:,None], axis=0) / max(wsum, 1e-30)
    X0 = XY - xc
    C = (X0*w[:,None]).T @ X0 / max(wsum, 1e-30)
    evals, evecs = np.linalg.eigh(C)
    maj = unit(evecs[:, np.argmax(evals)])
    ang = (np.degrees(np.arctan2(maj[1], maj[0])) % 180.0)
    return maj, ang

def mass_map_arbitrary(M_weights, XY_kpc, half_width_kpc, nbin):
    """Projected-mass map in ARBITRARY units (sum of weights per pixel)."""
    L = half_width_kpc
    x = np.clip(XY_kpc[:,0], -L, L); y = np.clip(XY_kpc[:,1], -L, L)
    H, xe, ye = np.histogram2d(x, y, bins=nbin, range=[[-L,L],[-L,L]], weights=M_weights)
    return H.T, xe, ye, (2*L)/nbin

def rms_z(arr_xz):
    z = arr_xz[:,2]
    return float(np.sqrt(np.mean((z - np.mean(z))**2)))

def read_cutout_particles(h5_path):
    with h5py.File(h5_path, "r") as f:
        stars = {
            "Coordinates": np.asarray(f["/PartType4/Coordinates"][...], dtype=np.float64),
            "Masses":      np.asarray(f["/PartType4/Masses"][...],      dtype=np.float64),
            "Velocities":  np.asarray(f["/PartType4/Velocities"][...],  dtype=np.float64),
        }
        gas = {}
        if "/PartType0/Coordinates" in f and "/PartType0/Masses" in f:
            gas["Coordinates"] = np.asarray(f["/PartType0/Coordinates"][...], dtype=np.float64)
            gas["Masses"]      = np.asarray(f["/PartType0/Masses"][...],      dtype=np.float64)
        else:
            gas["Coordinates"] = None
            gas["Masses"]      = None
    return stars, gas

# ────────────── Main ──────────────
def main():
    os.makedirs(OUT_DIR, exist_ok=True)

    # --- group catalog header + subhalo (source of truth for h, box, center, J_tot, rhalf) ---
    hdr = il.groupcat.loadHeader(GROUPCAT_BASE, SNAP)
    h            = float(hdr["HubbleParam"])
    box_ckpch    = float(hdr["BoxSize"])

    sh = il.groupcat.loadSingle(GROUPCAT_BASE, SNAP, subhaloID=SID)
    if sh is None: raise RuntimeError(f"Could not load subhalo {SID} from group catalog at {GROUPCAT_BASE}")

    center_ckpch          = np.array(sh["SubhaloPos"], float)
    rhalf_star_ckpch_cat  = float(sh["SubhaloHalfmassRadType"][4])  # stars
    J_tot                 = np.array(sh["SubhaloSpin"], float)      # catalog, total
    print(f"[CAT] h={h:.6f}  Box={box_ckpch:.1f} ckpc/h  center={center_ckpch}  rhalf(star)={rhalf_star_ckpch_cat:.3f} ckpc/h")

    # --- read particles from cutout ---
    stars, gas = read_cutout_particles(CUTOUT_H5)
    Xs_ckpch = stars["Coordinates"]; Ms_1e10 = stars["Masses"]; Vs_kms = stars["Velocities"]
    if Xs_ckpch is None or Ms_1e10 is None or Vs_kms is None:
        raise RuntimeError("Cutout missing required stellar fields.")

    # recenter by catalog center (ensures consistency with whole-snapshot calculations)
    Xs_rel_ckpch = recenter_positions(Xs_ckpch, center_ckpch, box_ckpch)

    # aperture based on catalog rhalf
    if use_rhalf_aperture and rhalf_star_ckpch_cat > 0:
        R = np.linalg.norm(Xs_rel_ckpch, axis=1)
        selS = (R <= rhalf_multiplier * rhalf_star_ckpch_cat)
        Xs_rel_ckpch, Ms_1e10, Vs_kms = Xs_rel_ckpch[selS], Ms_1e10[selS], Vs_kms[selS]
        print(f"[INFO] Aperture cut: rhalf*{rhalf_multiplier:.1f} = {(rhalf_multiplier*rhalf_star_ckpch_cat):.2f} ckpc/h | kept {len(Ms_1e10)} stars")

    # stellar bulk-velocity (optional)
    if REMOVE_BULK_VEL:
        vcm_star = (np.sum(Vs_kms * Ms_1e10[:,None], axis=0) / np.sum(Ms_1e10))
        Vs_rel = Vs_kms - vcm_star[None,:]
    else:
        Vs_rel = Vs_kms

    # stellar spin from cutout
    J_star = np.sum(np.cross(Xs_rel_ckpch, Vs_rel) * Ms_1e10[:,None], axis=0)

    # PCA v3 from stars
    Xs_kpc = Xs_rel_ckpch / h
    Ms_msun= Ms_1e10 * 1e10 / h
    evals3, evecs3, _ = pca3_weighted(Xs_kpc, Ms_msun)
    v3_hat = unit(evecs3[:,2])

    # diagnostics
    def ang(a,b): a=unit(a); b=unit(b); return np.degrees(np.arccos(np.clip(np.dot(a,b), -1, 1)))
    inc_t, PA_t = inc_PA_from_vector(J_tot)
    inc_s, PA_s = inc_PA_from_vector(J_star)
    maj2_xy, ang2_xy = pca2_weighted(Xs_kpc[:,[0,1]], Ms_msun)

    print(f"[INFO] Subhalo {SID} | N*={len(Ms_msun)}")
    print(f"  SubhaloSpin TOT    J = {J_tot}   | inc={inc_t:.2f}°, PA={PA_t:.2f}°")
    print(f"  Stellar-only  J*   = {J_star}    | inc={inc_s:.2f}°, PA={PA_s:.2f}°")
    print("  Angles between directions (deg):")
    print(f"    angle(J_tot,  J_star)  = {ang(J_tot, J_star):6.2f}")
    print(f"    angle(J_tot,  v3_pca)  = {ang(J_tot, v3_hat):6.2f}   (v3 ~ morphological normal)")
    print(f"    angle(J_star, v3_pca)  = {ang(J_star, v3_hat):6.2f}")
    print(f"  2D PCA major-axis angle in XY (deg, from +x): {ang2_xy:.2f} (expect ~ PA if inclined)")

    # ========= IMAGING (stars only by default, to mirror your TNG plot) =========
    if INCLUDE_GAS_IN_IMAGES and gas["Coordinates"] is not None and gas["Masses"] is not None:
        Xg_rel_ckpch = recenter_positions(gas["Coordinates"], center_ckpch, box_ckpch)
        if use_rhalf_aperture and rhalf_star_ckpch_cat > 0:
            Rg = np.linalg.norm(Xg_rel_ckpch, axis=1)
            selG = (Rg <= rhalf_multiplier * rhalf_star_ckpch_cat)
            Xg_rel_ckpch = Xg_rel_ckpch[selG]
            Mg_1e10      = gas["Masses"][selG]
        else:
            Mg_1e10 = gas["Masses"]
        Ximg_kpc  = np.vstack([Xs_rel_ckpch/h, Xg_rel_ckpch/h])
        Wimg_msun = np.concatenate([Ms_msun, (Mg_1e10 * 1e10 / h)])
        N_gas     = len(Mg_1e10)
    else:
        Ximg_kpc  = Xs_rel_ckpch / h    # stars only (matches reference)
        Wimg_msun = Ms_msun
        N_gas     = 0

    # rotations and maps
    L_kpc = (extent_kpc_h / 2.0) / h

    H_native, _, _, pix_native = mass_map_arbitrary(Wimg_msun, Ximg_kpc[:,[0,1]],   L_kpc, nbin)

    R_face_tot = rodrigues_u_to_v(unit(J_tot),  np.array([0,0,1.0]))
    R_face_str = rodrigues_u_to_v(unit(J_star), np.array([0,0,1.0]))
    R_face_pca = rodrigues_u_to_v(v3_hat,       np.array([0,0,1.0]))

    X_face_tot = (Ximg_kpc @ R_face_tot.T)
    X_face_str = (Ximg_kpc @ R_face_str.T)
    X_face_pca = (Ximg_kpc @ R_face_pca.T)

    H_face_t, _, _, _ = mass_map_arbitrary(Wimg_msun, X_face_tot[:,[0,1]], L_kpc, nbin)
    H_edge_t, _, _, _ = mass_map_arbitrary(Wimg_msun, X_face_tot[:,[0,2]], L_kpc, nbin)

    H_face_s, _, _, _ = mass_map_arbitrary(Wimg_msun, X_face_str[:,[0,1]], L_kpc, nbin)
    H_edge_s, _, _, _ = mass_map_arbitrary(Wimg_msun, X_face_str[:,[0,2]], L_kpc, nbin)

    H_face_p, _, _, _ = mass_map_arbitrary(Wimg_msun, X_face_pca[:,[0,1]], L_kpc, nbin)
    H_edge_p, _, _, _ = mass_map_arbitrary(Wimg_msun, X_face_pca[:,[0,2]], L_kpc, nbin)

    # thickness diagnostics (from edge-on maps)
    rms_edge_t = rms_z(X_face_tot)
    rms_edge_s = rms_z(X_face_str)
    rms_edge_p = rms_z(X_face_pca)
    print(f"[RMS z] edge-on (SubhaloSpin) = {rms_edge_t:.3f} kpc")
    print(f"[RMS z] edge-on (stellar)     = {rms_edge_s:.3f} kpc")
    print(f"[RMS z] edge-on (PCA)         = {rms_edge_p:.3f} kpc")

    # ========== PLOT (2 rows x 4 cols) ==========
    fig, axs = plt.subplots(2, 4, figsize=(22, 11), constrained_layout=True)

    def show_map(ax, H, title, xlabel="x [kpc]", ylabel="y [kpc]"):
        im = ax.imshow(np.log10(H + 1e-12), origin="lower",
                       extent=[-L_kpc, L_kpc, -L_kpc, L_kpc],
                       interpolation="nearest", cmap=cmap_name,
                       vmin=VMIN, vmax=VMAX)
        ax.set_xlabel(xlabel); ax.set_ylabel(ylabel); ax.set_title(title)
        for sp in ax.spines.values(): sp.set_edgecolor(fg_color)
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.02)
        cbar.set_label(r"$\log_{10}$ projected mass [arb.]")
        return im

    # Native XY with annotations (tot+stars)
    title0 = (
        f"Native XY | inc_tot={inc_t:.1f}°, PA_tot={PA_t:.1f}° | "
        f"inc_*={inc_s:.1f}°, PA_*={PA_s:.1f}° | sid={SID} | N*={len(Ms_msun)}"
        + (f" | N_gas={N_gas}" if INCLUDE_GAS_IN_IMAGES else "")
    )
    ax0 = axs[0,0]; show_map(ax0, H_native, title0)

    # projectile arrows and PA
    Jt_hat = unit(J_tot); Js_hat = unit(J_star)
    Jt_xy  = unit([Jt_hat[0], Jt_hat[1]])
    Js_xy  = unit([Js_hat[0], Js_hat[1]])

    n_tot = np.cross([0,0,1.0], Jt_hat)
    n_str = np.cross([0,0,1.0], Js_hat)
    n_tot_xy = unit([n_tot[0], n_tot[1]]) if np.linalg.norm(n_tot[:2])>0 else np.array([1.0,0.0])
    n_str_xy = unit([n_str[0], n_str[1]]) if np.linalg.norm(n_str[:2])>0 else np.array([1.0,0.0])

    PA_draw_tot = (np.degrees(np.arctan2(n_tot_xy[1], n_tot_xy[0])) % 360.0)
    PA_draw_str = (np.degrees(np.arctan2(n_str_xy[1], n_str_xy[0])) % 360.0)

    r = 0.55*L_kpc
    ax0.arrow(0,0, r*Jt_xy[0], r*Jt_xy[1], color=spin_tot_color,
              head_width=0.9*(2*L_kpc/nbin), head_length=1.6*(2*L_kpc/nbin), length_includes_head=True)
    ax0.text(r*Jt_xy[0]*1.02, r*Jt_xy[1]*1.02, "spin proj (total)", color=spin_tot_color, fontsize=10)

    ax0.arrow(0,0, r*Js_xy[0], r*Js_xy[1], color=spin_star_color,
              head_width=0.9*(2*L_kpc/nbin), head_length=1.6*(2*L_kpc/nbin), length_includes_head=True)
    ax0.text(r*Js_xy[0]*1.02, r*Js_xy[1]*1.02, "spin proj (stars)", color=spin_star_color, fontsize=10)

    ax0.arrow(0,0, r*n_tot_xy[0], r*n_tot_xy[1], color=nodes_color,
              head_width=0.9*(2*L_kpc/nbin), head_length=1.6*(2*L_kpc/nbin), length_includes_head=True, alpha=0.7)
    ax0.text(r*n_tot_xy[0]*1.02, r*n_tot_xy[1]*1.02, "line of nodes (tot)", color=nodes_color, fontsize=9, alpha=0.9)

    ax0.arrow(0,0, r*n_str_xy[0], r*n_str_xy[1], color=nodes_color,
              head_width=0.9*(2*L_kpc/nbin), head_length=1.6*(2*L_kpc/nbin), length_includes_head=True, linestyle='--', alpha=0.7)
    ax0.text(r*n_str_xy[0]*1.02, r*n_str_xy[1]*1.02, "line of nodes (stars)", color=nodes_color, fontsize=9, alpha=0.9)

    # PA arcs
    def draw_angle_arc(ax, radius, theta1_deg, theta2_deg, color="w", lw=2.0, nseg=100):
        t1 = np.radians(theta1_deg); t2 = np.radians(theta2_deg)
        ts = np.linspace(t1, t2, nseg)
        ax.plot(radius*np.cos(ts), radius*np.sin(ts), color=color, lw=lw)
    draw_angle_arc(ax0, 0.33*L_kpc, 0.0, PA_draw_tot, color=nodes_color, lw=2.0)
    draw_angle_arc(ax0, 0.27*L_kpc, 0.0, PA_draw_str, color=nodes_color, lw=1.5)

    # scale bar
    def draw_scale_bar(ax, half_width_kpc, frac=0.35, ypad_frac=0.06, color="w", lw=2.5):
        L = half_width_kpc; full = 2*L; bar_len = frac * full
        x0 = -L + 0.05*full; x1 = x0 + bar_len; y = -L + ypad_frac*full
        ax.plot([x0, x1], [y, y], color=color, lw=lw, solid_capstyle="butt")
        ax.text((x0+x1)/2, y - 0.035*full, f"{bar_len:.0f} kpc", color=color, ha="center", va="top", fontsize=10)
    draw_scale_bar(ax0, L_kpc, frac=0.35, color=scalebar_color)

    # Face-on / edge-on panels
    show_map(axs[0,1], H_face_t, "Face-on (SubhaloSpin → +z)")
    axs[0,1].plot([-0.6*L_kpc, 0.6*L_kpc], [0,0], color=axes_guide, lw=1.4, ls='--')
    axs[0,1].plot([0,0], [-0.6*L_kpc, 0.6*L_kpc], color=axes_guide, lw=1.4, ls='--')

    show_map(axs[0,2], H_edge_t, "Edge-on XZ (from SubhaloSpin → +z)", xlabel="x [kpc]", ylabel="z [kpc]")
    axs[0,2].plot([-0.75*L_kpc, 0.75*L_kpc], [0,0], color=axes_guide, lw=1.4, ls='--')

    show_map(axs[0,3], H_face_s, "Face-on (stellar spin → +z)")
    axs[0,3].plot([-0.6*L_kpc, 0.6*L_kpc], [0,0], color=axes_guide, lw=1.4, ls='--')
    axs[0,3].plot([0,0], [-0.6*L_kpc, 0.6*L_kpc], color=axes_guide, lw=1.4, ls='--')

    show_map(axs[1,0], H_edge_s, "Edge-on XZ (from stellar spin → +z)", xlabel="x [kpc]", ylabel="z [kpc]")
    axs[1,0].plot([-0.75*L_kpc, 0.75*L_kpc], [0,0], color=axes_guide, lw=1.4, ls='--')

    show_map(axs[1,1], H_face_p, "Face-on (PCA: v3 → +z)")
    axs[1,1].plot([-0.6*L_kpc, 0.6*L_kpc], [0,0], color=axes_guide, lw=1.4, ls='--')
    axs[1,1].plot([0,0], [-0.6*L_kpc, 0.6*L_kpc], color=axes_guide, lw=1.4, ls='--')

    show_map(axs[1,2], H_edge_p, "Edge-on XZ (from PCA v3 → +z)", xlabel="x [kpc]", ylabel="z [kpc]")
    axs[1,2].plot([-0.75*L_kpc, 0.75*L_kpc], [0,0], color=axes_guide, lw=1.4, ls='--')

    # Text panel
    ax_txt = axs[1,3]; ax_txt.axis("off")
    lines = [
        f"sid = {SID}",
        f"N* = {len(Ms_msun)}",
        (f"N_gas = {N_gas}" if INCLUDE_GAS_IN_IMAGES else "N_gas = 0 (images use stars only)"),
        "",
        f"inc_tot = {inc_t:.2f}°, PA_tot = {PA_t:.2f}°",
        f"inc_*   = {inc_s:.2f}°, PA_*   = {PA_s:.2f}°",
        "",
        f"∠(J_tot, J_*)   = {ang(J_tot, J_star):.2f}°",
        f"∠(J_tot, v3_PCA)= {ang(J_tot, v3_hat):.2f}°",
        f"∠(J_*, v3_PCA)  = {ang(J_star, v3_hat):.2f}°",
        f"PCA 2D major angle (XY) = {ang2_xy:.2f}°",
        "",
        f"RMS z (edge-on SubhaloSpin) = {rms_edge_t:.3f} kpc",
        f"RMS z (edge-on stellar)     = {rms_edge_s:.3f} kpc",
        f"RMS z (edge-on PCA)         = {rms_edge_p:.3f} kpc",
    ]
    ax_txt.text(0.02, 0.98, "\n".join(lines), transform=ax_txt.transAxes,
                va="top", ha="left", fontsize=11, color=fg_color)

    # Save figure
    outfile = os.path.join(OUT_DIR, f"debug_sid{SID}_part-{part}_snap{SNAP}.png")
    plt.savefig(outfile, dpi=dpi_out)
    plt.show()
    print(f"[SAVED] {outfile}")

    # Save tiny JSON + CSV with orientations
    orient_json = os.path.join(OUT_DIR, f"orientation_sid{SID}_snap{SNAP}.json")
    with open(orient_json, "w") as f:
        json.dump({
            "sid": SID, "snap": SNAP, "h": h,
            "center_ckpc_h": center_ckpch.tolist(),
            "rhalf_star_ckpc_h": rhalf_star_ckpch_cat,
            "J_tot": J_tot.tolist(),
            "J_star": J_star.tolist(),
            "v3_hat": v3_hat.tolist(),
            "inc_tot_deg": inc_t, "PA_tot_deg": PA_t,
            "inc_star_deg": inc_s, "PA_star_deg": PA_s,
            "PCA_major2D_XY_deg": float(ang2_xy),
        }, f, indent=2)
    print(f"[SAVED] {orient_json}")

    orient_csv = os.path.join(OUT_DIR, f"orientation_sid{SID}_snap{SNAP}.csv")
    with open(orient_csv, "w") as f:
        f.write("sid,snap,inc_tot_deg,PA_tot_deg,inc_star_deg,PA_star_deg,angle_Jtot_Jstar_deg,angle_Jtot_v3_deg,angle_Jstar_v3_deg,PCA_major2D_XY_deg\n")
        f.write(f"{SID},{SNAP},{inc_t:.6f},{PA_t:.6f},{inc_s:.6f},{PA_s:.6f},"
                f"{ang(J_tot,J_star):.6f},{ang(J_tot,v3_hat):.6f},{ang(J_star,v3_hat):.6f},{ang2_xy:.6f}\n")
    print(f"[SAVED] {orient_csv}")

if __name__ == "__main__":
    main()